In [34]:
import configparser
import pandas as pd
import re
import numpy as np
import string

#keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, LSTM, GlobalMaxPool1D, Bidirectional, Embedding, Dropout
from keras.models import Model

# sklearn
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

### Chargement des données

In [2]:
config = configparser.ConfigParser()
config.read('../config.cfg')
train = pd.read_csv(config['FILES']['TRAIN'])
test = pd.read_csv(config['FILES']['TEST'])

In [5]:
# Définition des différents labels disponibles sous forme de liste
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y = train[list_classes].values
list_sentences_train = train["comment_text"].fillna("_na_").values
list_sentences_test = test["comment_text"].fillna("_na_").values

### Creation du modèle Glove

In [90]:
def my_sentences(df):
    """
    Args: 
        df: dataframe a découper en phrases
        
    Cette fonction créé un fichier texte où toute la ponctuation a été retirée sauf les espaces et les :
    Le texte est mis en minuscule et les sauts de lignes dans un commentaires sont remplacés par un espace. 
    """
    corpus = open(config['FILES']['GLOVE_DIR'] + 'custom_corpus', 'a')
    for comment in df['comment_text']:
        comment = ' '.join(re.split('\n',comment.lower()))
        comment = re.sub(r'[^a-zA-Z0-9\s:]', '', comment)
        corpus.write(comment)

In [91]:
my_sentences(train)
my_sentences(test)

Pour créer le modèle Glove, on utilise les scripts bash mis à disposition par Glove sr https://github.com/stanfordnlp/GloVe
On créé des vecteurs de taille 300 pour se comparer aux autres modèles précédemment utilisés. On obtient un fichier text (vectors.txt) avec 1 ligne = 1 mot et son vecteur

In [8]:
EMBEDDING_FILE=config['FILES']['GLOVE_DIR'] + 'vectors.txt'

In [7]:
embed_size = 300 # taille du vecteur
max_features = 20000 # nombre de mots uniques utilisés (i.e nombre de lignes dans le vecteur d'embedding)
maxlen = 100 # nombre maximum de mots à considérer dans un commentaire

### Tokenisation pour la classification

In [9]:
# On garde les mots les plus fréquents dans le jeu d'entrainement pour établir notre liste de tokens.
tokenizer = Tokenizer(num_words=max_features)
# Calcul des mots les plus fréquents
tokenizer.fit_on_texts(list(list_sentences_train))
# Indexation des jeux d'entrainement et test, conversion du texte en séquence d'indexes
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
# Les commentaires ont une longueur variable qui peut être inférieure à maxlen
# On complète donc la séquence avec des 0, par défaut au début de la séquence
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

#### Lecture du fichier d'embedding

In [17]:
embeddings_index = dict()
for o in open(EMBEDDING_FILE):
    spt_o = o.strip().split()
    try:
        embeddings_index[spt_o[0]] = np.asarray(spt_o[1:], dtype='float32')
    except:
        pass

In [27]:
word_index = tokenizer.word_index
# le modèle pré-entrainé contient 3 millions de mots contre 20000 pour notre jeu de données.
vocabulary_size=min(len(word_index)+1,max_features)

# la matrice d'embedding représente le vocabulaire final à utiliser. 
# On ne gardera que 20000 vecteurs du modèle pré-entrainé. Leur longueur est de 300.
embedding_matrix = np.zeros((vocabulary_size, embed_size))


for word, i in word_index.items():
    if i>=max_features:
        continue
    try:
        embedding_vector = embeddings_index[word]
        if len(embedding_vector) == 300:
            embedding_matrix[i] = embedding_vector
        elif len(embedding_vector) > 300:
            embedding_matrix[i] = embedding_vector[0:300]
        else:
            while len(embedding_vector) < 300:
                embedding_vector.append(0)
    except KeyError:
        vec = np.zeros(embed_size)
        embedding_matrix[i]=vec

In [28]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 763s 5ms/step - loss: 0.0552 - acc: 0.9806 - val_loss: 0.0458 - val_acc: 0.9827
Epoch 2/2
143613/143613 [==============================] - 1026s 7ms/step - loss: 0.0408 - acc: 0.9844 - val_loss: 0.0455 - val_acc: 0.9832


In [30]:
test_label = pd.read_csv(config['FILES']['LABEL'])
test_label_strip = test_label[test_label.toxic != -1]
yt = test_label_strip[list_classes].values

In [31]:
model.evaluate(X_te[test_label_strip.index], yt, batch_size=1024)

63978/63978 [==============================] - 81s 1ms/step


[0.07264734107505243, 0.9698125831206363]

In [32]:
y_pred_t = model.predict(X_te, batch_size=1024)

In [35]:
print(accuracy_score(yt, y_pred_t[test_label_strip.index].round()))

0.8749257557285317


In [36]:
# Classification pour chaque label de façon indépendante
for label in range(0,6):
    print('... Traitement du label {}'.format(list_classes[label]))
    # On calcule l'accuracy des prédictions
    acc = accuracy_score(yt[:,label], y_pred_t[test_label_strip.index, label].round())
    print('L\'accuracy du jeu test est {}'.format(acc))
    print('Matrice de confusion sur le jeu test:')
    print(confusion_matrix(yt[:,label], y_pred_t[test_label_strip.index, label].round()))
    print('Rapport : ')
    print(classification_report(yt[:,label], y_pred_t[test_label_strip.index, label].round()) )

... Traitement du label toxic
L'accuracy du jeu test est 0.9181281065366219
Matrice de confusion sur le jeu test:
[[53406  4482]
 [  756  5334]]
Rapport : 
              precision    recall  f1-score   support

           0       0.99      0.92      0.95     57888
           1       0.54      0.88      0.67      6090

   micro avg       0.92      0.92      0.92     63978
   macro avg       0.76      0.90      0.81     63978
weighted avg       0.94      0.92      0.93     63978

... Traitement du label severe_toxic
L'accuracy du jeu test est 0.9940135671637125
Matrice de confusion sur le jeu test:
[[63534    77]
 [  306    61]]
Rapport : 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     63611
           1       0.44      0.17      0.24       367

   micro avg       0.99      0.99      0.99     63978
   macro avg       0.72      0.58      0.62     63978
weighted avg       0.99      0.99      0.99     63978

... Traitement du label obs